**Notes : **

1 - custom prediction python script should be provided (scoring.py)

2 - will check if artefacts can be installed to notebook

In [ ]:
# import required libraries
import os
import datetime
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azureml.core import (
    Experiment,
    Workspace,
    Run,
    Model,
)
from azure.identity import DefaultAzureCredential

In [ ]:
name = "train_model_credit_default_prediction"
latest_run_id = "" # gets the latest if empty

In [ ]:
workspace = Workspace.from_config()
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential)

In [8]:
# get experiment
experiment = Experiment(workspace, name)
# get experiment's latest run id
if latest_run_id:
    latest_run = Run.get_run(experiment, latest_run_id)
else:
    latest_run = next(experiment.get_runs())

['_Resource__source_path', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_arm_type', '_auto_increment_version', '_base_path', '_creation_context', '_from_container_rest_object', '_from_rest_object', '_get_arm_resource', '_get_arm_resource_and_params', '_id', '_is_anonymous', '_load', '_path', '_resolve_cls_and_type', '_serialize', '_source_path', '_to_arm_resource_param', '_to_dict', '_to_rest_object', '_update_path', '_version', 'base_path', 'creation_context', 'datastore', 'description', 'dump', 'flavors', 'id', 'job_name', 'latest_version', 'name', 'path', 'properties', 'tags', 'type', 'utc_time_created', 'version']
Help on method _load in m

In [ ]:
latest_run.download_files(prefix="model")

In [ ]:
# list downloaded artefacts
! ls model

In [ ]:
# model registration procedure
model = Model.register(model_path="model",
                          model_name=name,
                          description="description-here",
                          workspace=workspace)


In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
online_endpoint_name = "cv-model-" + datetime.datetime.now().strftime(
    "%m%d%H%M%f"
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="sample online endpoint for deploying model",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(online_endpoint_name)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

In [ ]:
deployment = ManagedOnlineDeployment(
    name=name,
    endpoint_name=online_endpoint_name,
    model=model.id,
    instance_type="Standard_DS3_V2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()